In [1]:
import os.path
import pandas as pd
import re
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import time 
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

import json
import time 
import string
import openpyxl
import os
import json, codecs
import warnings
warnings.filterwarnings("ignore")

In [2]:
base_url = 'https://dashboard.risika.com/home'
#base_url = 'https://dashboard.risika.com/' https://dashboard.risika.com/home
#Loading Selenium Webdriver 
driver= webdriver.Chrome()
wait = WebDriverWait(driver, 5)
company_list = pd.read_excel('Sweden_List.xlsx')

In [3]:
#for i in range(len(country_list)):
query_url = base_url #+ 'creditcheck/dk/'+ 14412107 #str(country_list['Code'][21]) + '/stores' #7,36,51
website_qurl_req = driver.get(query_url)
emailbox=driver.find_element(By.XPATH, '//*[@id="email"]')
time.sleep(1)
emailbox.clear()
time.sleep(1)
emailbox.send_keys("Hariprasad.Aleswaram@winora-group.de")
passbox = driver.find_element(By.XPATH, '//*[@id="password"]')
time.sleep(1)
passbox.send_keys("Hari@19091989")
#Clicking the location pointer directly
button=driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div/div[1]/div[2]/form/button')
driver.execute_script("arguments[0].click();", button)

In [4]:

# Create an empty DataFrame with specified columns
df = pd.DataFrame(columns=['Revenueresult_2021','Revenueresult_2020','Revenueresult_2019','Revenueresult_2018','Grossresult_2021','Grossresult_2020','Grossresult_2019','Grossresult_2018','P&L_2021','P&L_2020','P&L_2019','P&L_2018','Fixasset_2021','Fixasset_2020','Fixasset_2019','Fixasset_2018','Curasset_2021','Curasset_2020','Curasset_2019','Curasset_2018','Equity_2021','Equity_2020','Equity_2019','Equity_2018','TotalDebt_2021','TotalDebt_2020','TotalDebt_2019','TotalDebt_2018','Lib&Equity_2021','Lib&Equity_2020','Lib&Equity_2019','Lib&Equity_2018'])


# Wait for the dropdown to be visible
dropdown = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-cy="flag-dropdown-form-control"]'))
)

# Click the dropdown to open the options
dropdown.click()

# Wait for the country options to appear
country_options = WebDriverWait(driver, 10).until(
    EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'li.MuiMenuItem-root[data-cy^="flag-"]'))
)

# Find the desired country option and click it
desired_country = "SE"  # Change to the desired country code ("DK" for Denmark, "SE" for Sweden, etc.)
for option in country_options:
    if option.get_attribute("data-value") == desired_country:
        option.click()
        break


# Iterate over each company in the company_list
for i in range(len(company_list)):
    
    # Input data into the search box
    search_box = driver.find_element(By.CSS_SELECTOR, 'div[data-cy="search"] input.MuiInputBase-input')
    search_box.clear()
    search_box.send_keys(str(company_list['Momsnummer'][i]))
    time.sleep(1)
    # Press Enter to submit the search
    search_box.send_keys(Keys.ENTER)
    
    
    try:
        # Find the element using the first XPath
        element = driver.find_element_by_xpath('//*[@id="root"]/div[1]/main/div[2]/h2')
        text = element.text
    except NoSuchElementException:
        try:
            # Find the element using the new XPath
            element = driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/header/div/div[1]/div/form/div[3]/div')
            text = element.text.split('(')[1].split(')')[0]
            
        except NoSuchElementException:
            text = '1'

    # Check if the element contains the expected content
    if "Something went wrong" in text:
        
        print("Something went wrong Error for company id: ", company_list['Momsnummer'][i])
    elif '0' in text:
        element =  driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/header/div/div[1]/div/form/div[3]/div')
        text = element.text.split('(')[1].split(')')[0]
        print("Company id not found error for : ", company_list['Momsnummer'][i])
    else:
        # Wait until the general company information is present on the page
        general_info_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-cy="generalCompanyInformation"]')))
        # Find and click the button to reveal more information about company
        button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div[1]/main/div[2]/div[2]/div[1]/div[1]/div/div[2]/div[2]/div')))
        actions = ActionChains(driver)
        actions.move_to_element(button).perform()
        button.click()
        time.sleep(3)

        # Parse the page source using BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Find the general company information section
        results_company_information = soup.find("table", {"data-cy": "key-financial-figures-table"})#MuiStack-root css-wvp4lj 
        

        # Create a dictionary to store the extracted data for the current company
        data = {'Revenueresult_2021': None,'Revenueresult_2020': None,'Revenueresult_2019': None,'Revenueresult_2018': None,'Grossresult_2021': None,'Grossresult_2020': None,'Grossresult_2019': None,'Grossresult_2018': None,'P&L_2021': None,'P&L_2020': None,'P&L_2019': None,'P&L_2018': None,'Fixasset_2021': None,'Fixasset_2020': None,'Fixasset_2019': None,'Fixasset_2018': None,'Curasset_2021': None,'Curasset_2020': None,'Curasset_2019': None,'Curasset_2018': None,'Equity_2021': None,'Equity_2020': None,'Equity_2019': None,'Equity_2018': None,'TotalDebt_2021': None,'TotalDebt_2020': None,'TotalDebt_2019': None,'TotalDebt_2018': None,'Lib&Equity_2021': None,'Lib&Equity_2020': None,'Lib&Equity_2019': None,'Lib&Equity_2018': None}# 'Address': None, 'Phone': None, 'Established_On': None, 'Local_comp_id': None, 'Share_capital': None}
        
        #Revenue Resut
        try:
            data['Revenueresult_2021'] = results_company_information.find('h6', text='Revenue').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Revenueresult_2020'] = results_company_information.find('h6', text='Revenue').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Revenueresult_2019'] = results_company_information.find('h6', text='Revenue').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Revenueresult_2018'] = results_company_information.find('h6', text='Revenue').find_next('p').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        
        #Gross Resut
        try:
            data['Grossresult_2021'] = results_company_information.find('h6', text='Gross result').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Grossresult_2020'] = results_company_information.find('h6', text='Gross result').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Grossresult_2019'] = results_company_information.find('h6', text='Gross result').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Grossresult_2018'] = results_company_information.find('h6', text='Gross result').find_next('p').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        
        #Profit & Loss
        try:
            data['P&L_2021'] = results_company_information.find('h6', text='Profit loss').find_next('p').text
        except AttributeError:
            pass
        try:
            data['P&L_2020'] = results_company_information.find('h6', text='Profit loss').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['P&L_2019'] = results_company_information.find('h6', text='Profit loss').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['P&L_2018'] = results_company_information.find('h6', text='Profit loss').find_next('p').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        
        #Fixed Assets
        try:
            data['Fixasset_2021'] = results_company_information.find('h6', text='Fixed assets').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Fixasset_2020'] = results_company_information.find('h6', text='Fixed assets').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Fixasset_2019'] = results_company_information.find('h6', text='Fixed assets').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Fixasset_2018'] = results_company_information.find('h6', text='Fixed assets').find_next('p').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        
        #Current Assets
        
        try:
            data['Curasset_2021'] = results_company_information.find('h6', text='Current assets').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Curasset_2020'] = results_company_information.find('h6', text='Current assets').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Curasset_2019'] = results_company_information.find('h6', text='Current assets').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Curasset_2018'] = results_company_information.find('h6', text='Current assets').find_next('p').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        
        #Equity
        try:
            data['Equity_2021'] = results_company_information.find('h6', text='Equity').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Equity_2020'] = results_company_information.find('h6', text='Equity').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Equity_2019'] = results_company_information.find('h6', text='Equity').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Equity_2018'] = results_company_information.find('h6', text='Equity').find_next('p').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        
        #Total Debt
        
        try:
            data['TotalDebt_2021'] = results_company_information.find('h6', text='Total debt').find_next('p').text
        except AttributeError:
            pass
        try:
            data['TotalDebt_2020'] = results_company_information.find('h6', text='Total debt').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['TotalDebt_2019'] = results_company_information.find('h6', text='Total debt').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['TotalDebt_2018'] = results_company_information.find('h6', text='Total debt').find_next('p').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        
        #Liabilities & Equity
        
        try:
            data['Lib&Equity_2021'] = results_company_information.find('h6', text='Liabilities and equity').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Lib&Equity_2020'] = results_company_information.find('h6', text='Liabilities and equity').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Lib&Equity_2019'] = results_company_information.find('h6', text='Liabilities and equity').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
        try:
            data['Lib&Equity_2018'] = results_company_information.find('h6', text='Liabilities and equity').find_next('p').find_next('p').find_next('p').find_next('p').text
        except AttributeError:
            pass
       

        
        # Append the data dictionary as a new row in the DataFrame
        df = df.append(data, ignore_index=True)
        

# Quit the driver to close the browser
driver.quit()


In [5]:
df

,Revenueresult_2021,Revenueresult_2020,Revenueresult_2019,Revenueresult_2018,Grossresult_2021,Grossresult_2020,Grossresult_2019,Grossresult_2018,P&L_2021,P&L_2020,...,Equity_2019,Equity_2018,TotalDebt_2021,TotalDebt_2020,TotalDebt_2019,TotalDebt_2018,Lib&Equity_2021,Lib&Equity_2020,Lib&Equity_2019,Lib&Equity_2018
0,"458,967,287","435,476,668","390,704,280","433,844,986",-,-,-,-,"302,792","1,861,301",...,"22,060,146","21,572,354","170,477,236","130,819,621","161,933,580","143,544,565","242,894,009","205,324,745","224,892,722","200,760,355"
1,"96,417,455","85,042,606","77,660,921","75,112,445",-,-,-,-,"7,582,188","2,760,402",...,"11,467,093","12,271,815","29,249,237","30,838,073","24,496,939","27,410,496","53,648,679","55,416,399","43,719,071","46,847,017"
2,"4,332,764","6,116,862","4,362,635","4,225,114",-,-,-,-,"109,644","111,309",...,"738,764","712,081","4,032,876","3,219,866","3,788,754","2,747,745","5,531,589","4,661,730","5,103,541","4,118,703"
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [61]:
df.to_excel("Sweden_Risika_Revenue7.xlsx", index = False)